<a href="https://colab.research.google.com/github/yuxinl915/10701_proj_macro_var_selection/blob/main/UPDATED_lasso_variable_selection%2Bregression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import gc
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso

In [5]:
data = pd.read_csv("data.csv.gz")

In [6]:
firm_characteristics = ['me', 'be_me', 'ope_be', 'at_gr1', 'ret_3_1']
macro_variables = [col for col in data.columns if col not in firm_characteristics][2:]

print("Firm Characteristics:", firm_characteristics)
print("Macro Variables:", macro_variables)

Firm Characteristics: ['me', 'be_me', 'ope_be', 'at_gr1', 'ret_3_1']
Macro Variables: ['TB3MS', 'GS1', 'GS5', 'GS10', 'GS30', 'BAA', 'AAA', 'UNRATE', 'ICSA', 'M2', 'FEDFUNDS', 'VIX', 'OIL', 'INDPRO', 'TCU', 'RPI', 'PAYEMS', 'CPI', 'CPICORE', 'PCEPI', 'term_spread', 'def_spread', 'inflation', 'core_inflation', 'real_ff', 'm2_growth', 'indpro_growth', 'rpi_growth', 'payems_growth', 'oil_ret']


In [7]:
for firm_char in firm_characteristics:
    for macro_var in macro_variables:
        new_col_name = f"{firm_char}_x_{macro_var}"
        data[new_col_name] = data[firm_char] * data[macro_var]

/tmp/ipython-input-3621462998.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col_name] = data[firm_char] * data[macro_var]
/tmp/ipython-input-3621462998.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col_name] = data[firm_char] * data[macro_var]
/tmp/ipython-input-3621462998.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get 

In [ ]:
unscale_cols = ["eom", "ret_exc_lead1m"]
scale_cols = [c for c in data.columns if c not in unscale_cols]

In [ ]:
# 1. Sort by date and reset index so index == row position
data = data.sort_values("eom").reset_index(drop=True)

# 2. Find the row position where test period begins
cutoff_date = "2020-01-31"
test_idx = data.index[data["eom"] >= cutoff_date][0]  # now this is a position

# 3. Build train/test arrays directly from slices (no X_all / y_all)
X_train = data.loc[:test_idx-1, scale_cols].to_numpy(dtype="float32")
y_train = data.loc[:test_idx-1, "ret_exc_lead1m"].to_numpy(dtype="float32")

X_test  = data.loc[test_idx:, scale_cols].to_numpy(dtype="float32")
y_test  = data.loc[test_idx:, "ret_exc_lead1m"].to_numpy(dtype="float32")

# 4. Free the big DataFrame as soon as we're done with it
del data
gc.collect()


23

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [ ]:

# -----------------------------
# 4. Time-series LASSO with modest CV (to save RAM)
# -----------------------------
tscv = TimeSeriesSplit(n_splits=3)  # fewer splits = less memory and time
lasso_ts = LassoCV(
    alphas=np.logspace(-5, 0, 100),
    cv=tscv,
    max_iter=5000,
    n_jobs=1,
    random_state=42
)

lasso_ts.fit(X_train, y_train)
print("alpha* =", lasso_ts.alpha_)
print("Number of nonzero coefficients:", np.sum(lasso_ts.coef_ != 0))

C:\Users\yimengs\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.851e+01, tolerance: 1.536e+00
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\yimengs\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.450e+01, tolerance: 1.536e+00
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\yimengs\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the feature

alpha* = 0.0004641588833612782
Number of nonzero coefficients: 63


C:\Users\yimengs\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.387e+04, tolerance: 3.011e+03
  model = cd_fast.enet_coordinate_descent(


In [ ]:
coefs = lasso_ts.coef_
nonzero = [(name, coef) for name, coef in zip(scale_cols, coefs) if coef != 0]
for name, coef in nonzero:
    print(f"{name:25s}  {coef: .5f}")

be_me                       0.80156
GS1                         0.94956
GS5                        -1.34766
GS30                        0.39509
BAA                         0.28318
UNRATE                     -0.17074
ICSA                        0.01629
FEDFUNDS                   -0.15521
VIX                        -0.11631
OIL                         0.02209
TCU                         0.02081
RPI                        -0.04473
PAYEMS                      0.04417
term_spread                 0.11816
def_spread                  0.03047
inflation                   0.08024
core_inflation             -0.02217
real_ff                    -0.03189
m2_growth                  -0.00834
indpro_growth               0.07275
rpi_growth                 -0.01876
payems_growth              -0.04783
oil_ret                    -0.00064
me_x_UNRATE                 0.00457
me_x_M2                     0.00420
me_x_FEDFUNDS              -0.00316
me_x_VIX                    0.01003
me_x_def_spread            -

In [ ]:
print("alpha* =", enet.alpha_)
print("largest alpha in grid =", lasso_ts.alphas_[0])
print("smallest alpha in grid =", lasso_ts.alphas_[-1])

alpha* = 0.0044040077361278615
largest alpha in grid = 1.0
smallest alpha in grid = 1e-05


In [ ]:
lasso_final = Lasso(alpha=lasso_ts.alpha_, max_iter=5000)
lasso_final.fit(X_train, y_train)

y_pred_test = lasso_final.predict(X_test)

ic_test  = np.corrcoef(y_test, y_pred_test)[0, 1]
mse_test = np.mean((y_test - y_pred_test)**2)
r2_test  = 1 - mse_test / np.var(y_test)

print("Test IC:", ic_test)
print("Test MSE:", mse_test)
print("Test R^2:", r2_test)

Test IC: 0.01973286167808728
Test MSE: 20.50855
Test R^2: -289.23211669921875


C:\Users\yimengs\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.387e+04, tolerance: 3.011e+03
  model = cd_fast.enet_coordinate_descent(


In [1]:
survival_factors = [
    "be_me",
    "GS1",
    "GS5",
    "GS30",
    "BAA",
    "UNRATE",
    "ICSA",
    "FEDFUNDS",
    "VIX",
    "OIL",
    "TCU",
    "RPI",
    "PAYEMS",
    "term_spread",
    "def_spread",
    "inflation",
    "core_inflation",
    "real_ff",
    "m2_growth",
    "indpro_growth",
    "rpi_growth",
    "payems_growth",
    "oil_ret",
    "me_x_UNRATE",
    "me_x_M2",
    "me_x_FEDFUNDS",
    "me_x_VIX",
    "me_x_def_spread",
    "me_x_inflation",
    "me_x_indpro_growth",
    "me_x_rpi_growth",
    "me_x_payems_growth",
    "be_me_x_TB3MS",
    "be_me_x_GS1",
    "be_me_x_GS5",
    "be_me_x_GS10",
    "be_me_x_GS30",
    "be_me_x_BAA",
    "be_me_x_AAA",
    "be_me_x_UNRATE",
    "be_me_x_M2",
    "be_me_x_FEDFUNDS",
    "be_me_x_VIX",
    "be_me_x_OIL",
    "be_me_x_INDPRO",
    "be_me_x_RPI",
    "be_me_x_PAYEMS",
    "be_me_x_CPI",
    "be_me_x_CPICORE",
    "be_me_x_term_spread",
    "be_me_x_def_spread",
    "be_me_x_inflation",
    "be_me_x_core_inflation",
    "be_me_x_m2_growth",
    "be_me_x_indpro_growth",
    "be_me_x_rpi_growth",
    "be_me_x_payems_growth",
    "ret_3_1_x_GS1",
    "ret_3_1_x_GS5",
    "ret_3_1_x_VIX",
    "ret_3_1_x_term_spread",
    "ret_3_1_x_inflation",
    "ret_3_1_x_oil_ret"
]

In [8]:
keep_cols = ["eom", "ret_exc_lead1m"] + survival_factors
data_reduced = data[keep_cols]


In [ ]:
data_reduced.to_csv("survival_data.csv.gz", index=False, compression="gzip")
